In [9]:
import pandas as pd

data_files = {
    "Categories": "../basededatos/categories.csv",
    "Customers": "../basededatos/customers.csv",
    "Employees": "../basededatos/employees.csv",
    "Orders": "../basededatos/orders.csv",
    "Order details": "../basededatos/order_details.csv",
    "Products": "../basededatos/products.csv",
    "Suppliers": "../basededatos/suppliers.csv"
}

categories = pd.read_csv(data_files["Categories"], header=0, index_col=False)
customers = pd.read_csv(data_files["Customers"], header=0, index_col=False)
employees = pd.read_csv(data_files["Employees"], header=0, index_col=False)
orders = pd.read_csv(data_files["Orders"], header=0, index_col=False)
order_details = pd.read_csv(data_files["Order details"], header=0, index_col=False)
products = pd.read_csv(data_files["Products"], header=0, index_col=False)
suppliers = pd.read_csv(data_files["Suppliers"], header=0, index_col=False)


ventas.csv

orderid,fecha,empleado,producto,idproducto,cantidad,preccioventa,descuento,importe,categoria,proveedor,ciudad_cliente,pais_cliente,cliente

In [10]:
def merge_dataframes(orders, employees, order_details, products, categories, suppliers, customers):
    return (
        orders.merge(employees, on='EmployeeID', how='left')
        .merge(order_details, on='OrderID', how='inner')
        .merge(products, on='ProductID', how='left')
        .merge(categories, on='CategoryID', how='left')
        .merge(suppliers, on='SupplierID', how='left')
        .merge(customers, on='CustomerID', how='left')
    )

def calculate_importe(result):
    return ((result['UnitPrice_x'] - (result['Discount'] * result['UnitPrice_x'])) * result['Quantity']).round(2)

merged_result = merge_dataframes(orders, employees, order_details, products, categories, suppliers, customers)

merged_result['importe'] = calculate_importe(merged_result)

final_result = merged_result[[
    'OrderID',
    'OrderDate',
    'FirstName',
    'LastName',
    'ProductName',
    'ProductID',
    'Quantity',
    'UnitPrice_x',
    'Discount',
    'importe',
    'CategoryName',
    'CompanyName_x',
    'City',
    'Country',
    'ContactName_y'
]].rename(columns={
    'OrderID': 'orderid',
    'OrderDate': 'fecha',
    'FirstName': 'empleado',
    'LastName': 'Apellido',  
    'ProductName': 'producto',
    'ProductID': 'idproducto',
    'Quantity': 'cantidad',
    'UnitPrice_x': 'preccioventa',
    'Discount': 'descuento',
    'importe': 'importe',
    'CategoryName': 'categoria',
    'CompanyName_x': 'proveedor',
    'City': 'ciudad_cliente',
    'Country': 'pais_cliente',
    'ContactName_y': 'cliente'
})

display(final_result)

,orderid,fecha,empleado,Apellido,producto,idproducto,cantidad,preccioventa,descuento,importe,categoria,proveedor,ciudad_cliente,pais_cliente,cliente
0,10248,1996-07-04,Steven,Buchanan,Queso Cabrales,11,12,14.0,0.00,168.00,Dairy Products,Cooperativa de Quesos 'Las Cabras',Reims,France,Paul Henriot
1,10248,1996-07-04,Steven,Buchanan,Singaporean Hokkien Fried Mee,42,10,9.8,0.00,98.00,Grains/Cereals,Leka Trading,Reims,France,Paul Henriot
2,10248,1996-07-04,Steven,Buchanan,Mozzarella di Giovanni,72,5,34.8,0.00,174.00,Dairy Products,Formaggi Fortini s.r.l.,Reims,France,Paul Henriot
3,10249,1996-07-05,Michael,Suyama,Tofu,14,9,18.6,0.00,167.40,Produce,Mayumi's,Münster,Germany,Karin Josephs
4,10249,1996-07-05,Michael,Suyama,Manjimup Dried Apples,51,40,42.4,0.00,1696.00,Produce,"G'day, Mate",Münster,Germany,Karin Josephs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10284,1996-08-19,Margaret,Peacock,Schoggi Schokolade,27,15,35.1,0.25,394.88,Confections,Heli Süßwaren GmbH & Co. KG,Frankfurt a.M.,Germany,Renate Messner
96,10284,1996-08-19,Margaret,Peacock,Gula Malacca,44,21,15.5,0.00,325.50,Condiments,Leka Trading,Frankfurt a.M.,Germany,Renate Messner
97,10284,1996-08-19,Margaret,Peacock,Camembert Pierrot,60,20,27.2,0.25,408.00,Dairy Products,Gai pâturage,Frankfurt a.M.,Germany,Renate Messner
98,10284,1996-08-19,Margaret,Peacock,Laughing Lumberjack Lager,67,5,11.2,0.25,42.00,Beverages,Bigfoot Breweries,Frankfurt a.M.,Germany,Renate Messner


ventas_tiempos.csv

cliente,fecha_venta,fecha_venta_anterior,diferencia_dias,diferencia_meses,monto_venta

In [46]:
def group_order_details(result):
    grouped_result = (
        result
        .groupby(
            ['ContactName', 'OrderDate'],  
            as_index=False
        )
        .agg(
            fecha_venta_anterior=('OrderDate_anterior', 'first'),
            diferencia_dias=('diferencia_dias', 'first'),
            diferencia_meses=('diferencia_meses', 'first'),
            monto_venta=('monto_venta', 'sum')  
        )
    )
    return grouped_result

def main(orders, customers, order_details, products):
    orders = convert_order_dates(orders)
    
    result = merge_dataframes(orders, customers, order_details, products)

    result['OrderDate_anterior'] = result.apply(lambda row: get_previous_order_date(row, orders), axis=1)
    
    result = calculate_differences_and_amount(result)

    final_result = group_order_details(result)

    final_result.rename(columns={
        'OrderDate': 'fecha_venta',
        'OrderDate_anterior': 'fecha_venta_anterior',
        'ContactName': 'cliente'
    }, inplace=True)

    columns_to_keep = ['cliente', 'fecha_venta', 'fecha_venta_anterior', 'diferencia_dias', 'diferencia_meses', 'monto_venta']
    final_result = final_result[columns_to_keep]
    
    final_result = final_result.sort_values(by='fecha_venta')

    return final_result

final_result = main(orders, customers, order_details, products)

display(final_result)

,cliente,fecha_venta,fecha_venta_anterior,diferencia_dias,diferencia_meses,monto_venta
25,Paul Henriot,1996-07-04,NaT,NaN,0,440.000
15,Karin Josephs,1996-07-05,NaT,NaN,0,1863.400
21,Mary Saveley,1996-07-08,NaT,NaN,0,654.060
19,Mario Pontes,1996-07-08,NaT,NaN,0,1552.600
24,Pascale Cartrain,1996-07-09,NaT,NaN,0,3597.900
20,Mario Pontes,1996-07-10,1996-07-08,2.0,0,1444.800
37,Yang Wang,1996-07-11,NaT,NaN,0,556.620
22,Michael Holz,1996-07-12,NaT,NaN,0,2490.500
27,Paula Parente,1996-07-15,NaT,NaN,0,517.800
6,Carlos Hernández,1996-07-16,NaT,NaN,0,1119.900
